# 超参调节、批量标准化、编程框架

## 1. 超参调节

### 1.1 调参过程

调参的优先级：

1）学习速率 $\alpha$

2）如果使用动量梯度下降 $\beta$；隐藏神经元数量；微批大小。

3）神经网络的层数；学习速率衰减

4）如果使用Adam，几乎不太会去调 $\beta_1=0.9, \beta_2=0.999, \epsilon=10^{-8}$

在机器学习的早期阶段，通常会使用网格搜索。当超参较少的情况下，网格搜索是没问题的。对于神经网络这样超参众多的模型，更推荐随机挑选。因为通常很难预先知道哪一个超参会更重要，实际上超参发挥的作用，会有很大的差距，会重要的超参尝试更多的值搜索效果会更好。

![Try random values, don't use a grid](img/Try random values, don't use a grid.png)

另一个推荐的搜索策略，是采用由粗到细的搜索（coarse to fine），在表现较好的超参区域，放大，进一步搜索。

![Coarse to fine](img/Coarse to fine.png)

### 1.2 使用合适的刻度来搜索超参

上一节提到，随机挑选超参可以提高超参搜索的效率，但与此同时，随机抽样并不意味着随机均匀抽样。当超参搜索的范围量级变动较大时，使用合适的刻度来随机抽样十分重要。

![Appropriate scale for hyperparameters](img/Appropriate scale for hyperparameters.png)

![Hyperparameters for exponentially weighted averages](img/Hyperparameters for exponentially weighted averages.png)

### 1.3 超参调节实践：熊猫策略 vs 鱼子酱策略

不同类型的问题，超参通常都是重新调节的；即使对于同一个问题，随着时间推移，新的数据进入训练集，时不时对模型表现进行评估并重新搜索超参也是十分必要的。

取决于所拥有的计算资料，调节超参也包含串行和并行两种策略。

![Pandas vs Caviar](img/Pandas vs Caviar.png)

## 2. 批量标准化

### 2.1 对神经网络中的激活值进行标准化

批量标准化可以使超参搜索更容易，使神经网络模型更健壮，也使得训练更深的神经网络更为容易。批量标准化的起源，是当我们对输入进行标准化后，训练过程会更快。基于同样的原因，如果对每个隐藏层的激活值也进行标准化，那么训练过程会不会进一步加快？但在技术实现中，**实际情况更多是对线性组合的Z值进行标准化，而非激活值**。

![Normalizing inputs to speed up learning](img/Normalizing inputs to speed up learning.png)

批量标准化，并不像对输入的处理，将其都标准化为均值0，标准差1，而是允许不同的均值和标准差。这个是通过 $\gamma$ 和 $\beta$ 这两个值来控制的，这两个值像权重一样，是模型可以学习得到的。

![Implementing Batch Norm](img/Implementing Batch Norm.png)

### 2.2 将批量标准化应用到神经网络中

![Adding Batch Norm to a network](img/Adding Batch Norm to a network.png)

批量标准化通常和微批梯度下降一起使用。另外，在使用了批量标准化之后，参数 $b$ 实际上可以去除了。

![Working with mini-batches](img/Working with mini-batches.png)

### 2.3 理解批量标准化

1. 和对输入进行标准化一样，可以加快模型训练速度。

2. 能更好地应对数据集输入分布的偏移。这种偏移，术语叫做covariate shift。它描述的是，算法已经学到了从X到Y的映射关系，当输入X的分布发生变化，我们可能需要重新训练模型。不论X->Y的真实函数没有发生变化，重新训练模型都是必要的。
![Learning on shifting input distribution](img/Learning on shifting input distribution.png)
![Why this is a problem with neural network](img/Why this is a problem with neural network.png)

3. 类似于Dropout，批量标准化同时也起到正则化的效果。（但不要将其作为正则化的手段来用）
![Batch Norm as regularization](img/Batch Norm as regularization.png)

### 2.4 针对测试集的批量标准化

批量标准化在训练过程中，每次处理一个微批。而在针对测试集进行预测的时候，可能会需要对单个样本进行计算，这时会缺少微批的均值和方差。我们需要单独预测均值和方差。可以使用各个批次的均值和方差，使用指数加权移动平均的方式，来进行预测。

![Batch Norm at test time](img/Batch Norm at test time.png)

## 3. 多元分类

### 3.1 Softmax回归

Softmax回归是Logistic回归的推广形式，可以用来处理多元分类问题。将神经网络最后一层，替换为Softmax回归层，即可进行多元分类。

![Recognizing cats, dogs, and baby chicks](img/Recognizing cats, dogs, and baby chicks.png)

Softmax回归本身的决策边界是线性的。

![Softmax examples](img/Softmax examples.png)

### 3.2 训练Softmax分类器

Softmax的概念是和Hardmax对应的，Hardmax会产出一个类似[1, 0, 0, 0]的向量，而Softmax产生的向量，是和为1的概率表述向量。

Softmax的损失函数，反向传播，略。